### Modelo Convolusión 2D - Keras

#### Importación de librerias para la generación del modelo

In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import preprocess_input

#### Se cargan los datos del csv de las imágenes

In [5]:
train_df = pd.read_csv('boneage-training-dataset.csv')
test_df = pd.read_csv('boneage-test-dataset.csv')

train_df['id'] = train_df['id'].apply(lambda x: str(x)+'.png')
train_df['gender'] = train_df['male'].apply(lambda x: 'male' if x else 'female')

#### Se separa el training data en Training y Validation

In [6]:
df_train, df_valid = train_test_split(train_df, test_size = 0.2, random_state = 0)

#### Se preparan los sets para las imagenes para Train, Validation y Testing

In [7]:
img_size = 256

train_data_generator = ImageDataGenerator(preprocessing_function = preprocess_input)
val_data_generator = ImageDataGenerator(preprocessing_function = preprocess_input)

train_generator = train_data_generator.flow_from_dataframe(
    dataframe = df_train, directory = 'boneage-training-dataset/boneage-training-dataset',
    x_col= 'id', y_col= 'boneage', batch_size = 32, seed = 42,
    shuffle = True, class_mode= 'raw', flip_vertical = True,
    color_mode = 'rgb', target_size = (img_size, img_size))

validation_generator = val_data_generator.flow_from_dataframe(
    dataframe = df_valid, directory = 'boneage-training-dataset/boneage-training-dataset',
    x_col= 'id', y_col= 'boneage', batch_size = 32, seed = 42,
    shuffle = True, class_mode= 'raw', flip_vertical = True,
    color_mode = 'rgb', target_size = (img_size, img_size))

test_data_generator = ImageDataGenerator(preprocessing_function = preprocess_input)

test_generator = test_data_generator.flow_from_directory(
    directory = 'boneage-test-dataset',
    shuffle = False, 
    class_mode = None,
    color_mode = 'rgb',
    target_size = (img_size,img_size))

Found 10088 validated image filenames.
Found 2523 validated image filenames.
Found 200 images belonging to 1 classes.


#### Se valida la existencia de las imagenes y se transforman a modo de prueba

In [8]:
test_X, test_Y = next(val_data_generator.flow_from_dataframe( 
                            dataframe = df_valid, directory = 'boneage-training-dataset/boneage-training-dataset',
                            x_col = 'id', y_col = 'boneage', 
                            target_size = (img_size, img_size),
                            batch_size = 2523,
                            class_mode = 'raw'
                            ))

Found 2523 validated image filenames.


#### Generación del modelo y entrenamiento

In [9]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout


model = Sequential()
# Input Layer
model.add(Conv2D(16, kernel_size=(7,7), strides=(2,2), activation='relu', input_shape=(img_size, img_size, 3)))
# Convulotional Layers
model.add(Conv2D(32, kernel_size=(5,5), strides=(2,2), activation='relu'))
# Flattening
model.add(Flatten())
# Dense Layer
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
history = model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=15)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
315/315 [==============================] - 478s 2s/step - loss: 1860.6343 - mse: 1860.6343 - val_loss: 1451.5078 - val_mse: 1451.5078
Epoch 2/15
315/315 [==============================] - 475s 2s/step - loss: 1398.8809 - mse: 1398.8809 - val_loss: 1620.4595 - val_mse: 1620.4595
Epoch 3/15
315/315 [==============================] - 471s 1s/step - loss: 1220.8604 - mse: 1220.8604 - val_loss: 1193.9430 - val_mse: 1193.9430
Epoch 4/15
315/315 [==============================] - 482s 2s/step - loss: 1043.0212 - mse: 1043.0212 - val_loss: 1337.1180 - val_mse: 1337.1180
Epoch 5/15
315/315 [==============================] - 482s 2s/step - loss: 921.6007 - mse: 921.6007 - val_loss: 1141.9824 - val_mse: 1141.9824
Epoch 6/15
315/315 [==============================] - 478s 2s/step - loss: 801.6290 - mse: 801.6290 - val_loss: 1090.3987 - val_mse: 1090.3987
Epoch 7/15
315/315 [==============================] - 476s

#### Generación de las predicciones con el set de validación

In [9]:
predictions = model.predict(validation_generator)

#### Se realiza una validación para revisar que tan bien funcionó el modelo

In [10]:
print("Edad Osea de la predicción:",predictions[0][0],"meses")
print("Edad Osea real:",df_valid['boneage'].iloc[0],"meses")


Edad Osea de la predicción: 184.60518 años
Edad Osea real: 94 años


In [11]:
fd = open('document.txt','a')
for i in range(2523):
    fd.write('\n'+str(df_valid['boneage'].iloc[i]) + ',' + str(predictions[i][0]))
fd.close()

#### Guardar modelo

In [10]:
model.save('modelConvolution2DKeras.h5') 

#### Cargando el modelo en una nueva variable

In [12]:
 # Recreate the exact same model, including its weights and the optimizer
load_model = tf.keras.models.load_model('modelConvolution2DKeras.h5')

# Show the model architecture
load_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 125, 125, 16)      2368      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 32)        12832     
_________________________________________________________________
flatten (Flatten)            (None, 119072)            0         
_________________________________________________________________
dense (Dense)                (None, 10)                1190730   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 1,205,941
Trainable params: 1,205,941
Non-trainable params: 0
_________________________________________________________________


In [13]:
load_predictions = load_model.predict(validation_generator)

In [21]:
print("Edad Osea de la predicción:",load_predictions[0][0],"meses")
print("Edad Osea real:",df_valid['boneage'].iloc[0],"meses")

Edad Osea de la predicción: 205.00821 meses
Edad Osea real: 94 meses
